In [1]:
!pip install --upgrade pip
!pip install pymongo
!pip install -U nltk
!pip install joblib
!pip install pandas
!pip install sklearn

    100% |████████████████████████████████| 1.4MB 3.8MB/s 
  Found existing installation: pip 19.0.2
    Uninstalling pip-19.0.2:
      Successfully uninstalled pip-19.0.2
    100% |████████████████████████████████| 409kB 1.6MB/s 
    100% |████████████████████████████████| 1.4MB 2.6MB/s 
  Stored in directory: /home/jovyan/.cache/pip/wheels/4b/c8/24/b2343664bcceb7147efeb21c0b23703a05b23fcfeaceaa2a1e
Successfully built nltk
    100% |████████████████████████████████| 286kB 7.0MB/s 
    100% |████████████████████████████████| 10.1MB 200kB/s 
    100% |████████████████████████████████| 512kB 2.5MB/s 
    100% |████████████████████████████████| 17.3MB 209kB/s 
    100% |████████████████████████████████| 5.4MB 260kB/s 
    100% |████████████████████████████████| 24.8MB 361kB/s 
  Stored in directory: /home/jovyan/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [21]:
from pymongo import MongoClient
import pandas as pd
import re
import nltk

from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from joblib import dump, load
from sklearn.model_selection import cross_val_score

#Download do corpus da nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
def _connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]

In [4]:
def read_mongo(db, collection, query={}, host='ds249824.mlab.com', port='49824', username='app', password='nodeapp01', no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df

In [5]:
acidente = read_mongo('labeling_zone','tweets',query={'label':'Sim'})
nao_acidente = read_mongo('labeling_zone','tweets',query={'label':'Não'})
#full_data = acidente.append(nao_acidente[0:170])
full_data = acidente.append(nao_acidente)

full_data.head()

,coordinates,data,geo,hashtags,id,label,labeled_by,lang,location,place,quoted,retweeted,texto,urls,user_mentions,usuario
0,None,Tue Feb 19 08:06:49 +0000 2019,None,[],1097769458460954624,Sim,5c7e98f8d363db2425775f24,pt,São Paulo,None,False,False,As faixas foram liberadas na Rod Dom Pedro I (...,"[{'url': 'https://t.co/IpIWplGWqp', 'expanded_...",[],radiotransitofm
1,None,Tue Feb 19 08:10:15 +0000 2019,None,[],1097770324278546432,Sim,5c7e98f8d363db2425775f24,pt,"São Paulo/SP, Brasil",None,False,False,Um dos caminhões e a carreta tombaram no acide...,"[{'url': 'https://t.co/bgvms9jsTK', 'expanded_...",[],TransitoSampaSP
2,None,Thu Feb 21 09:11:13 +0000 2019,None,"[{'text': 'acidente', 'indices': [17, 26]}]",1098510441767161856,Sim,5c7e98f8d363db2425775f24,pt,São Paulo,None,False,False,Ouvintes relatam #acidente no km 39 da rodovia...,[],[],radiotransitofm
3,None,Wed Feb 20 18:45:23 +0000 2019,None,[],1098292549234671616,Sim,5c7e98f8d363db2425775f24,pt,São Paulo/SP - Brasil,None,False,False,"13h24 - Acidente de trânsito, carro x moto, na...","[{'url': 'https://t.co/OO7X33NXye', 'expanded_...",[],BombeirosPMESP
4,None,Wed Feb 20 18:46:15 +0000 2019,None,[],1098292765224566784,Sim,5c7e98f8d363db2425775f24,pt,"São Paulo/SP, Brasil",None,False,False,"Acidente de trânsito, carro x moto, na R.Pedro...","[{'url': 'https://t.co/LNXwM9R7WR', 'expanded_...","[{'screen_name': 'BombeirosPMESP', 'name': '19...",TransitoSampaSP


In [7]:
print(len(acidente))
print(len(nao_acidente))
full_data = acidente.append(nao_acidente[0:277])

277
1499


In [8]:
def remove_urls(text):
    return re.sub(r'https?:\/\/.*[\r\n]*', '', text)

def to_lower(text):
    return text.lower()

In [9]:
full_data['texto_formatado'] = full_data['texto'].apply(lambda t: remove_urls(str(t)))
full_data['texto_formatado'] = full_data['texto_formatado'].apply(lambda t: to_lower(t))

In [10]:
tokenizer = TreebankWordTokenizer()
full_data['tokens'] = full_data['texto_formatado'].apply(lambda t: tokenizer.tokenize(t))
full_data[['texto_formatado','tokens']].head()

,texto_formatado,tokens
0,as faixas foram liberadas na rod dom pedro i (...,"[as, faixas, foram, liberadas, na, rod, dom, p..."
1,um dos caminhões e a carreta tombaram no acide...,"[um, dos, caminhões, e, a, carreta, tombaram, ..."
2,ouvintes relatam #acidente no km 39 da rodovia...,"[ouvintes, relatam, #, acidente, no, km, 39, d..."
3,"13h24 - acidente de trânsito, carro x moto, na...","[13h24, -, acidente, de, trânsito, ,, carro, x..."
4,"acidente de trânsito, carro x moto, na r.pedro...","[acidente, de, trânsito, ,, carro, x, moto, ,,..."


In [11]:
def create_stopword_list():
    portuguese_stops = set(stopwords.words('portuguese'))
    portuguese_stops.add('rt')

    with open('punctuation.txt','r+') as punct_file:
        puncts = punct_file.readlines()

    for item in puncts:    
        portuguese_stops.add(item.strip())
        
    return portuguese_stops

stop_w = create_stopword_list()

#aplica a remocao de stop-words
full_data['words'] = full_data['tokens'].apply(lambda w: [word for word in w if word not in stop_w]) 
#exibe resultado intermediario
full_data[['tokens','words']].head(10)

,tokens,words
0,"[as, faixas, foram, liberadas, na, rod, dom, p...","[faixas, liberadas, rod, dom, pedro, i, sp-065..."
1,"[um, dos, caminhões, e, a, carreta, tombaram, ...","[caminhões, carreta, tombaram, acidente, deixa..."
2,"[ouvintes, relatam, #, acidente, no, km, 39, d...","[ouvintes, relatam, acidente, km, 39, rodovia,..."
3,"[13h24, -, acidente, de, trânsito, ,, carro, x...","[13h24, acidente, trânsito, carro, x, moto, ru..."
4,"[acidente, de, trânsito, ,, carro, x, moto, ,,...","[acidente, trânsito, carro, x, moto, r.pedro, ..."
5,"[bloqueio, total, do, acesso, do, #, rodoanel,...","[bloqueio, total, acesso, rodoanel, sentido, p..."
6,"[ouvintes, relatam, #, acidente, no, km, 39, d...","[ouvintes, relatam, acidente, km, 39, rodovia,..."
7,"[capotamento, no, #, corredornortesul, sentido...","[capotamento, corredornortesul, sentido, santa..."
8,"[capotamento, no, #, corredornortesul, sentido...","[capotamento, corredornortesul, sentido, santa..."
9,"[capotamento, no, #, corredornortesul, sentido...","[capotamento, corredornortesul, sentido, santa..."


In [12]:
stemer= SnowballStemmer(language='portuguese')
full_data['stem_words'] = full_data['words'].apply(lambda t: [stemer.stem(word) for word in t])
full_data[['words','stem_words']].head()

,words,stem_words
0,"[faixas, liberadas, rod, dom, pedro, i, sp-065...","[faix, liber, rod, dom, pedr, i, sp-065, km, 3..."
1,"[caminhões, carreta, tombaram, acidente, deixa...","[caminhõ, carret, tomb, acident, deix, cair, p..."
2,"[ouvintes, relatam, acidente, km, 39, rodovia,...","[ouvint, relat, acident, km, 39, rodov, bandei..."
3,"[13h24, acidente, trânsito, carro, x, moto, ru...","[13h24, acident, trânsit, carr, x, mot, rua, p..."
4,"[acidente, trânsito, carro, x, moto, r.pedro, ...","[acident, trânsit, carr, x, mot, r.pedr, toled..."


In [13]:
def encode_label(text):
    if text=='Sim':
        return 1
    elif text=='Não':
        return 0
    else:
        return -1

full_data['target'] = full_data['label'].apply(lambda label: encode_label(label))
full_data['target'].describe()

count    554.000000
mean       0.500000
std        0.500452
min        0.000000
25%        0.000000
50%        0.500000
75%        1.000000
max        1.000000
Name: target, dtype: float64

In [14]:
full_data['clean_text'] = full_data['stem_words'].apply(lambda t: str(' '.join(t)))

count_vect = CountVectorizer()
X_TF = count_vect.fit_transform(full_data['clean_text'])
print(X_TF.shape)

(554, 1991)


In [15]:
print(X_TF.toarray()) 

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [16]:
tfidf_transformer = TfidfTransformer()
X = tfidf_transformer.fit_transform(X_TF)
X.shape

(554, 1991)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, full_data['target'], test_size=0.33, random_state=42)

In [18]:
nb = MultinomialNB().fit(X_train, y_train)
svm = svm.SVC().fit(X_train,y_train)
tree = tree.DecisionTreeClassifier().fit(X_train,y_train)
logr = LogisticRegression().fit(X_train,y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [26]:
nbc = MultinomialNB()
svmc = svm.SVC()
treec = tree.DecisionTreeClassifier()
logr = LogisticRegression()

<h1> Realiza o cross validation </h1>

In [27]:
nb_scores = cross_val_score(nbc, X, full_data['target'], cv=5)
svm_scores = cross_val_score(svmc, X, full_data['target'], cv=5)
tree_scores = cross_val_score(treec, X, full_data['target'], cv=5)
logr_scores = cross_val_score(logr, X, full_data['target'], cv=5)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [28]:
print(nb_scores)
print(svm_scores)
print(tree_scores)
print(logr_scores)

[0.83035714 0.82142857 0.80909091 0.86363636 0.67272727]
[0.67857143 0.77678571 0.70909091 0.67272727 0.52727273]
[0.89285714 0.83035714 0.86363636 0.9        0.84545455]
[0.875      0.875      0.9        0.92727273 0.77272727]


In [19]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

def print_metrics(y_true,y_predicted):
    acc = accuracy_score(y_test.values,y_predicted)
    print('Acurácia: ' + str(acc))
    #accuracy_score(y_true, y_predicted)
    print('Matriz de Confusão:' )
    print(confusion_matrix(y_true, y_predicted, labels=[0, 1]))
    print('Recall')
    print(recall_score(y_true, y_predicted, average='macro')  )
    print('Precision')
    print(precision_score(y_true, y_predicted, average='macro')  )

In [20]:
print('Naive Bayes')
print_metrics(y_test,nb.predict(X_test))
print('\n')
print('Decision Tree')
print_metrics(y_test,tree.predict(X_test))
print('\n')
print('Suport Vector Machines')
print_metrics(y_test,svm.predict(X_test))
print('\n')
print('Logistic Regressor')
print_metrics(y_test,logr.predict(X_test))

Naive Bayes
Acurácia: 0.825136612021858
Matriz de Confusão:
[[73 19]
 [13 78]]
Recall
0.8253105590062111
Precision
0.8264804603212659


Decision Tree
Acurácia: 0.8852459016393442
Matriz de Confusão:
[[81 11]
 [10 81]]
Recall
0.8852723363592929
Precision
0.8852723363592929


Suport Vector Machines
Acurácia: 0.4972677595628415
Matriz de Confusão:
[[ 0 92]
 [ 0 91]]
Recall
0.5
Precision
0.24863387978142076


Logistic Regressor
Acurácia: 0.907103825136612
Matriz de Confusão:
[[81 11]
 [ 6 85]]
Recall
0.9072503583373148
Precision
0.9082255747126436


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
